In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2
%env TF_FORCE_GPU_ALLOW_GROWTH=true

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2
env: TF_FORCE_GPU_ALLOW_GROWTH=true


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
structures = pd.read_pickle("datasets/structures_defects.pickle.gzip")

In [4]:
# TODO(inner perfectionist) eval is unsecure
defects = pd.read_csv(
  "datasets/dichalcogenides_innopolis_202105/descriptors.csv", index_col="_id",
  converters={"cell": eval, "defects": eval})

In [5]:
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

import tensorflow as tf
import numpy as np

In [6]:
import wandb
from wandb.keras import WandbCallback

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# TODO(kazeevn) pass structure as the global state
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [9]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [10]:
config = wandb.config
config.target = "formation_energy"

In [11]:
from typing import Union, List, Dict
from megnet.data.graph import StructureGraphFixedRadius
from pymatgen.analysis.local_env import NearNeighbors
from megnet.data.graph import Converter
class ZAwareStrucutreGraph(StructureGraphFixedRadius):
    def __init__(
        self,
        nn_strategy: Union[str, NearNeighbors] = "MinimumDistanceNNAll",
        atom_converter: Converter = None,
        bond_converter: Converter = None,
        cutoff: float = 5.0,
    ):
      raise NotImplemented("Doesn't work due to difficult estate planning")
      self.cutoff = cutoff
      super().__init__(
              nn_strategy=nn_strategy, atom_converter=atom_converter, bond_converter=bond_converter, cutoff=self.cutoff
      )
        
      @staticmethod
      def get_atom_features(structure):
        return [[i.species.Z, i.coords[2]] for i in structure.sites]

In [12]:
nfeat_edge = 10
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 15, nfeat_edge), 0.5),
                  cutoff=15)
model = MEGNetModel(nfeat_edge=nfeat_edge, nfeat_global=2, graph_converter=gc, npass=2)

In [13]:
scaler = StandardScaler.from_training_data(train.defect_representation,
                                           train[config.target], is_intensive=False)
model.target_scaler = scaler

In [14]:
model.train(train.defect_representation, train[config.target],
            validation_structures=test.defect_representation,
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=2000, verbose=1, patience=1000)

Epoch 1/2000


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_bond/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of m

21/21 [==============================] - 17s 47ms/step - loss: 1.4131


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 2.84860, saving model to callback/val_mae_00001_2.848598.hdf5


Epoch 2/2000
21/21 [==============================] - 1s 46ms/step - loss: 0.8184
Epoch 3/2000
21/21 [==============================] - 1s 45ms/step - loss: 0.8891
Epoch 4/2000
21/21 [==============================] - 1s 46ms/step - loss: 0.8167
Epoch 5/2000
21/21 [==============================] - 1s 46ms/step - loss: 0.6560
Epoch 6/2000
21/21 [==============================] - 1s 47ms/step - loss: 0.3815


...

21/21 [==============================] - 1s 63ms/step - loss: 0.1675
Epoch 1551/2000
21/21 [==============================] - 1s 63ms/step - loss: 0.1498
Epoch 1552/2000
21/21 [==============================] - 1s 66ms/step - loss: 0.1754
Epoch 1553/2000
21/21 [==============================] - 1s 68ms/step - loss: 0.1513
Epoch 1554/2000
21/21 [==============================] - 1s 66ms/step - loss: 0.2174
Epoch 1555/2000
21/21 [==============================] - 1s 71ms/step - loss: 0.1665
Epoch 1556/2000
21/21 [==============================] - 2s 71ms/step - loss: 0.1975
Epoch 1557/2000
21/21 [==============================] - 1s 67ms/step - loss: 0.1540
Epoch 1558/2000
21/21 [==============================] - 1s 64ms/step - loss: 0.1026
Epoch 1559/2000
21/21 [==============================] - 1s 67ms/step - loss: 0.1382
Epoch 1560/2000
21/21 [==============================] - 1s 67ms/step - loss: 0.2883
Epoch 1561/2000
21/21 [==============================] - 1s 65ms/step - loss: 0.1

21/21 [==============================] - 1s 49ms/step - loss: 0.1100
Epoch 1647/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1435
Epoch 1648/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1249
Epoch 1649/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1293
Epoch 1650/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.0696
Epoch 1651/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.0619
Epoch 1652/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.0899
Epoch 1653/2000
21/21 [==============================] - 1s 50ms/step - loss: 0.1509
Epoch 1654/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1761
Epoch 1655/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0933
Epoch 1656/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.2102
Epoch 1657/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.2

21/21 [==============================] - 1s 62ms/step - loss: 0.1827
Epoch 1743/2000
21/21 [==============================] - 1s 59ms/step - loss: 0.1614
Epoch 1744/2000
21/21 [==============================] - 1s 59ms/step - loss: 0.2691
Epoch 1745/2000
21/21 [==============================] - 1s 63ms/step - loss: 0.1742
Epoch 1746/2000
21/21 [==============================] - 1s 67ms/step - loss: 0.2953
Epoch 1747/2000
21/21 [==============================] - 1s 69ms/step - loss: 0.1986
Epoch 1748/2000
21/21 [==============================] - 1s 60ms/step - loss: 0.1452
Epoch 1749/2000
21/21 [==============================] - 1s 64ms/step - loss: 0.2336
Epoch 1750/2000
21/21 [==============================] - 1s 66ms/step - loss: 0.1254
Epoch 1751/2000
21/21 [==============================] - 1s 67ms/step - loss: 0.1232
Epoch 1752/2000
21/21 [==============================] - 1s 64ms/step - loss: 0.3974
Epoch 1753/2000
21/21 [==============================] - 1s 65ms/step - loss: 0.1

21/21 [==============================] - 1s 49ms/step - loss: 0.2162
Epoch 1839/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1675
Epoch 1840/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1007
Epoch 1841/2000
21/21 [==============================] - 1s 50ms/step - loss: 0.1188
Epoch 1842/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1407
Epoch 1843/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1274
Epoch 1844/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.0803
Epoch 1845/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1268
Epoch 1846/2000
21/21 [==============================] - 1s 50ms/step - loss: 0.1152
Epoch 1847/2000
21/21 [==============================] - 1s 50ms/step - loss: 0.2432
Epoch 1848/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.1238
Epoch 1849/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.1

21/21 [==============================] - 1s 48ms/step - loss: 0.0628
Epoch 1935/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0903
Epoch 1936/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0469
Epoch 1937/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0997
Epoch 1938/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0748
Epoch 1939/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0935
Epoch 1940/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0439
Epoch 1941/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.0555
Epoch 1942/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0407
Epoch 1943/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0201
Epoch 1944/2000
21/21 [==============================] - 1s 49ms/step - loss: 0.0744
Epoch 1945/2000
21/21 [==============================] - 1s 48ms/step - loss: 0.0

In [15]:
model.save_model("models/MEGNet-defect-only/defect_E")

INFO:tensorflow:Assets written to: models/MEGNet-defect-only/defect_E/assets


INFO:tensorflow:Assets written to: models/MEGNet-defect-only/defect_E/assets
